# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [13]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [14]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)
     
    def forward(self, x):
        in_size = x.size(0) # one batch
        # x: 64*10*12*12
        x = F.relu(self.mp(self.conv1(x)))
        # x: 64*20*4*4
        x = F.relu(self.mp(self.conv2(x)))
        # x: 64*320
        x = x.view(in_size, -1) # flatten the tensor
        # x: 64*10
        x = self.fc(x)
        return F.log_softmax(x)

model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


Next, we can start to train and evaluate!

In [15]:
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()
        output = model(images)
        loss = F.nll_loss(output, labels)
        loss.backward()
        optimizer.step()
print('Train finished.')
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
train_acc=0
test_acc=0
for images, labels in tqdm(train_loader):
    output = model(images)
    pred = output.argmax(dim=1, keepdim=True)  
    train_acc += pred.eq(labels.view_as(pred)).sum().item()
    
for images, labels in tqdm(test_loader):
    output = model(images)
    pred = output.argmax(dim=1, keepdim=True)  
    test_acc += pred.eq(labels.view_as(pred)).sum().item()

  1%|          | 3/468 [00:00<00:16, 28.43it/s]

Train finished.


100%|██████████| 78/78 [00:02<00:00, 36.70it/s]


#### Q5:
Please print the training and testing accuracy.

In [16]:
print('\nTrain set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_acc, len(train_loader.dataset),
        100. * train_acc / len(train_loader.dataset)))

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_acc, len(test_loader.dataset),
        100. * test_acc / len(test_loader.dataset)))  


Train set: Accuracy: 58972/60000 (98%)


Test set: Accuracy: 9822/10000 (98%)

